In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pickle

In [2]:
path = 'PetImages/'

In [3]:
def get_vgg_model():
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(2)
    preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
    base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                               include_top=False,
                                               weights='imagenet')
    base_model.trainable = False
    inputs = tf.keras.Input(shape=(128,128,3))
    x = inputs
    x = preprocess_input(x)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [4]:
def get_images(category):
    temp = []
    temp_file = [name for name in os.listdir(path+f'{category}')]
    for i in tqdm(temp_file):
        try:
            t = Image.open(path+f'/{category}/{i}')        
            t = np.asarray(t.resize((128,128)))
            if t.shape == (128,128,3):
                temp.append(np.asarray(t))            
        except:
            pass    
    return temp

In [5]:
def dump_pickle(dat):
    with open('data.pickle', 'wb') as handle:
        pickle.dump(dat, handle)
def load_pickle(path):
    with open(path,'rb') as handle:
        temp = pickle.load(handle)
    return temp

In [6]:
def pre_process():
    cat = get_images("Cat")
    dog = get_images("Dog")
    x = np.array(cat+dog)
    y1 = ([0 for i in range(len(cat))])
    y2 = ([1 for i in range(len(dog))])
    y = np.array(y1 + y2)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    dump_pickle((X_train, X_test, y_train, y_test))
    return X_train, X_test, y_train, y_test    

In [7]:
X_train, X_test, y_train, y_test = pre_process()

 91%|████████████████████████████████████████████████████████████████████▌      | 11434/12501 [01:09<00:07, 147.11it/s]C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting t

In [8]:
model = get_vgg_model()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

9412608/9406464 [==============================] - 1s 0us/step


In [9]:
history = model.fit(X_train, y_train, epochs = 5,validation_split = 0.2)

Epoch 1/5
418/418 [==============================] - 90s 215ms/step - loss: 0.1140 - accuracy: 0.9555 - val_loss: 0.0698 - val_accuracy: 0.9731
Epoch 2/5
418/418 [==============================] - 90s 214ms/step - loss: 0.0691 - accuracy: 0.9744 - val_loss: 0.0753 - val_accuracy: 0.9716
Epoch 3/5
418/418 [==============================] - 84s 201ms/step - loss: 0.0618 - accuracy: 0.9763 - val_loss: 0.0671 - val_accuracy: 0.9767
Epoch 4/5
418/418 [==============================] - 82s 195ms/step - loss: 0.0545 - accuracy: 0.9808 - val_loss: 0.0652 - val_accuracy: 0.9758
Epoch 5/5
418/418 [==============================] - 80s 192ms/step - loss: 0.0530 - accuracy: 0.9804 - val_loss: 0.0850 - val_accuracy: 0.9701


In [10]:
model.evaluate(X_test,y_test)

258/258 [==============================] - 42s 163ms/step - loss: 0.0856 - accuracy: 0.9702


[0.08555962145328522, 0.970223605632782]

In [11]:
model.save('./saved_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_model\assets


In [22]:
t = X_train[0]
output = list(model.predict(t.reshape(1,t.shape[0],t.shape[1],t.shape[2]))[0])
label = output.index(max(output))
if label == 0:
    print("Cat")
else:
    print("Dog")

Dog


In [18]:
y_train[0]

1